## Model 2: Here we are taking just the new features and run them through an xgboost model 

In [3]:
import xgboost as xgb
import pandas as pd

# Loading dataset here
df = pd.read_csv('combined_df.csv')

# Features and label used
feature_cols = ['IsPolitical', 'TalksAboutOwnParty', 'TalksAboutOtherParty', 
                'IsAccusatory', 'IsHeSaidSheSaid', 'IsQuote']

X = df[feature_cols]
y = df['binary_label']


In [4]:
print(df.columns.tolist())

['label', 'statement', 'IsPolitical', 'TalksAboutOwnParty', 'TalksAboutOtherParty', 'IsAccusatory', 'IsHeSaidSheSaid', 'IsQuote', 'binary_label']


In [5]:
# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Training the model here XGBoost
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

c:\Users\Leona\anaconda3\envs\mlproj\lib\site-packages\xgboost\training.py:183: UserWarning: [20:58:23] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [7]:
#saving the model

# Save the trained XGBoost model to a file
model.save_model('xgboost_new_feature_model.json')


In [8]:
# Predict
probs = model.predict_proba(X_test)[:, 1]

In [9]:
# Evaluate
from sklearn.metrics import accuracy_score

y_pred = (probs > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f"XGBoost Accuracy: {accuracy:.4f}")

#save results
section2_results = pd.DataFrame({
    'statement': df.loc[X_test.index, 'statement'].values,
    'label': y_test.values,
    'predicted_percentage_true': probs
})

section2_results.to_csv('xgboost_ollama_predictions.csv', index=False)

XGBoost Accuracy: 0.5913


In [10]:
#printing the metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")


Accuracy : 0.5913
Precision: 0.6114
Recall   : 0.8592
F1 Score : 0.7144
